In [1]:
#import stuffz
import nltk
import numpy as np
import pandas as pd

In [8]:
from nltk.metrics import edit_distance

In [36]:
import re

In [2]:
crosscheck_df = pd.read_csv('cross_check_test.csv')

In [3]:
crosscheck_df.head()

,id,status,record_id,source_type,source_gender,source_title,source_date,source_publisher,source_pub_viaf,source_genre,...,target_type,target_gender,target_title,target_date,target_publisher,target_pub_viaf,target_genre,target_price,target_author_viaf,target_author_name
0,152,done,136129795,single_focus,m,Times Before the Reformation,1880.0,Robert Carter & Brothers,NaN,historical nonfiction,...,single_focus,m,"Times before the Reformation, with an account ...",1880.0,New York: Robert Carter & Brothers,NaN,Nonfiction,NaN,http://viaf.org/viaf/93207229,William Dinwiddie
1,662,done,124766315,single_focus,f,One Man Who Was Content,1897.0,The Century Company,NaN,fiction,...,single_focus,f,One Man Who Was Content,1897.0,New York: The Century Company,NaN,Fiction,$1.00,http://viaf.org/viaf/44455381,Schuyler Van Rensselaer
2,236,done,124905465,multi,NaN,NaN,NaN,NaN,NaN,NaN,...,multi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,660,done,136243777,multi,multi,NaN,NaN,NaN,NaN,NaN,...,cluster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,250,done,136680305,not_review,NaN,NaN,NaN,NaN,NaN,NaN,...,not_review,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Goals:
+ Calculate interrater reliability
+ Type: single_focus vs. not single_focus
+ Title, author, & publisher: levenshtein distance, disregard punc and capitalization
+ VIAF: just the number

will add other stuff as i remember

In [7]:
crosscheck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 23 columns):
id                    35 non-null int64
status                35 non-null object
record_id             35 non-null int64
source_type           35 non-null object
source_gender         18 non-null object
source_title          17 non-null object
source_date           17 non-null float64
source_publisher      17 non-null object
source_pub_viaf       0 non-null float64
source_genre          17 non-null object
source_price          10 non-null object
source_author_viaf    15 non-null object
source_author_name    17 non-null object
target_type           35 non-null object
target_gender         17 non-null object
target_title          17 non-null object
target_date           17 non-null float64
target_publisher      17 non-null object
target_pub_viaf       0 non-null float64
target_genre          17 non-null object
target_price          10 non-null object
target_author_viaf    17 non-null ob

In [31]:
source_ex = crosscheck_df.source_title[0]
target_ex = crosscheck_df.target_title[0]

In [38]:
' '.join(re.split('[^a-zA-Z]', target_ex))

'Times before the Reformation  with an account of fra Girolamo Savonarolam the Friar of Florence'

In [39]:
source_ex = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', source_ex) if w.isalpha()])
target_ex = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', target_ex) if w.isalpha()])

In [40]:
source_ex

'times before the reformation'

In [41]:
target_ex

'times before the reformation with an account of fra girolamo savonarolam the friar of florence'

In [43]:
(source_ex in target_ex) or (target_ex in source_ex)

True

In [46]:
def flagSimilarity(source, target):
    """
    Cleans punc and lowercases. If source/target are same but one includes extra text, returns a yellow flag.
    Otherwise if source/target have edit_distance > 3, returns a red flag.
    """
    source = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', source) if w.isalpha()])
    target = ' '.join([w.lower() for w in re.split('[^a-zA-Z]', target) if w.isalpha()])
    if source == target:
        return 'green'
    elif (source_ex in target_ex) or (target_ex in source_ex):
        return 'yellow'
    elif edit_distance > 3:
        return 'red'

In [47]:
flagSimilarity(crosscheck_df.source_title[0], crosscheck_df.target_title[0])

'yellow'

In [55]:
def compReviewType(source_type, target_type):
    if source_type == target_type:
        return 'green'
    elif ((source_type == 'not_review' and target_type != 'not_review') or 
          (target_type == 'not_review' and source_type != 'not_review')):
        return 'red'
    elif ((source_type == 'single_focus' and target_type != 'single_focus') or
          (target_type == 'single_focus' and source_type != 'single_focus')):
        return 'red'
    else: return 'green'

In [56]:
compReviewType(crosscheck_df.source_type[0], crosscheck_df.target_type[0])

'green'